In [1]:
#   Instalación de librerías
!pip install tweepy
!pip install textblob

In [2]:
import tweepy
import time
import matplotlib.pyplot as plt
from textblob import TextBlob

In [3]:
###  Definición de variables
#consumer_key = "****DYFh3A"
#consumer_secret = "****DYFh3A"

#access_token = "1882464041827786758-d3hDuLx3Bed9h7NzT2QBZen4njjSda"
#access_token_secret = "LJ2M1EJQwrO1Br8xSy1iuogFHT9XXtjRece8cKngMZmT6"

# Claves de acceso
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAEK3yQEAAAAAk1dXWsGpz4O%2FpcWGBzozFbQfUY4%3DgTzqNPG7ZLWj2f4nqMe9AIJVy0lR7rsZT4ls39Ci9XIkgcnuG7"

In [4]:
def buscar_tweets_relevantes(palabra, numero_tweets, lenguaje):
    """
    Busca tweets públicos más relevantes que contengan una palabra clave,
    con soporte para paginación y manejo del límite de la API.

    Args:
        palabra (str): Palabra clave para buscar.
        numero_tweets (int): Número máximo de tweets a devolver.
        lenguaje (str): Idioma de los tweets ('en' para inglés, 'es' para español, etc.).

    Returns:
        list: Lista de diccionarios con los textos de los tweets encontrados.
    """
    try:
        # Configurar el cliente de Twitter
        client = tweepy.Client(bearer_token=BEARER_TOKEN)
        
        # Construir la consulta de búsqueda
        query = f"{palabra} lang:{lenguaje} -is:retweet"  # Excluir retweets
        
        # Configuración inicial
        tweets = []
        max_results_per_request = 50  # Reducido para evitar problemas de límite
        next_token = None

        while len(tweets) < numero_tweets:
            remaining_tweets = numero_tweets - len(tweets)
            try:
                # Realizar solicitud a la API
                response = client.search_recent_tweets(
                    query=query,
                    max_results=min(max_results_per_request, remaining_tweets),
                    tweet_fields=["text", "created_at", "author_id"],
                    sort_order="relevancy",  # Ordenar por relevancia
                    next_token=next_token
                )
                
                # Si no hay más datos, salir del bucle
                if not response.data:
                    break
                
                # Agregar tweets a la lista
                tweets.extend(response.data)
                
                # Obtener el token para la siguiente página
                next_token = response.meta.get("next_token")
                if not next_token:
                    break

                # Pausar para evitar alcanzar el límite de la API
                time.sleep(2)  # Esperar 2 segundos entre solicitudes

            except tweepy.TooManyRequests:
                print("Límite de solicitudes alcanzado. Esperando 15 minutos...")
                time.sleep(900)  # Esperar 15 minutos y reintentar
        
        # Formatear resultados (extraer texto)
        resultados = [{"texto": tweet.text, "autor_id": tweet.author_id, "fecha": tweet.created_at} for tweet in tweets]
        print(f"Se encontraron {len(resultados)} tweets.")
        return resultados

    except Exception as e:
        print(f"Error al buscar tweets: {e}")
        return []

In [5]:
def analizar_polaridad_tweets(tweets):
    """
    Analiza la polaridad de los tweets utilizando TextBlob.

    Args:
        tweets (list): Lista de diccionarios con los textos de los tweets.

    Returns:
        tuple: Tres elementos:
            - Lista con las polaridades de los tweets.
            - Lista con los textos de los tweets analizados.
            - Número total de tweets analizados.
    """
    polaridades = []
    textos = []

    for tweet in tweets:
        texto = tweet["texto"]
        analisis = TextBlob(texto)  # Analiza el texto del tweet
        polaridad = analisis.sentiment.polarity  # Obtiene la polaridad (-1 a 1)
        
        polaridades.append(polaridad)
        textos.append(texto)

    total_tweets = len(textos)
    return polaridades, textos, total_tweets

In [6]:
def graficar_polaridades(polaridades, textos):
    """
    Genera un scatterplot de las polaridades de los tweets y muestra un cajetín con la polaridad promedio.

    Args:
        polaridades (list): Lista de valores de polaridad (-1 a 1).
        textos (list): Lista de textos de los tweets (solo para referencia en el eje X).
    """
    # Calcular la polaridad promedio
    polaridad_promedio = sum(polaridades) / len(polaridades) if polaridades else 0
    
    # Crear el scatterplot
    plt.figure(figsize=(10, 6))
    plt.scatter(range(len(polaridades)), polaridades, color="blue", alpha=0.7, label="Polaridad de Tweets")
    
    # Etiquetas y título
    plt.title("Análisis de Polaridad de Tweets", fontsize=16)
    plt.xlabel("Índice de Tweet", fontsize=12)
    plt.ylabel("Polaridad (-1 a 1)", fontsize=12)
    plt.axhline(y=0, color="gray", linestyle="--", linewidth=0.8, label="Neutral (0)")
    
    # Cajetín con polaridad promedio
    cajetin_texto = f"Polaridad Promedio: {polaridad_promedio:.2f}"
    props = dict(boxstyle="round", facecolor="lightyellow", alpha=0.8)
    plt.text(0.05, 0.95, cajetin_texto, transform=plt.gca().transAxes, fontsize=12,
             verticalalignment="top", bbox=props)
    
    # Leyenda y mostrar gráfico
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
if __name__ == "__main__":
    # Parámetros para buscar tweets
    palabra_clave = "IA"
    numero_tweets = 10000  # Reducido para evitar límites de API
    idioma = "es"  # Idioma: 'es' (español), 'en' (inglés), etc.
    
    # Buscar tweets
    resultados = buscar_tweets_relevantes(palabra_clave, numero_tweets, idioma)
    
    # Analizar polaridad de los tweets
    polaridades, textos, total_tweets = analizar_polaridad_tweets(resultados)
    
    # Graficar polaridades
    graficar_polaridades(polaridades, textos)

Límite de solicitudes alcanzado. Esperando 15 minutos...
